In [5]:
from difflib import SequenceMatcher
from asrtoolkit import cer, wer
import cv2
import pytesseract
import numpy as np
from statistics import mean

In [6]:
def img2text(fileName, process = True):
    img = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
    if process:
        img = processImg(img)

    return pytesseract.image_to_string(img)

In [7]:
def processImg(img):
    return img

In [21]:
def calcCER(begin, end, path = 'test_images/captured/grayscale/..._CC_GRAY.jpg'):
    res = []
    for i in range(begin, end + 1):
        fileName = str(i).zfill(3)
        imageName = path.replace('...', fileName)
        truthName = f'test_images/truth/{fileName}_OCR_ASCII_TEXT_GT.txt'

        text = img2text(imageName)
        with open(truthName, 'r') as f:
            groundTruth = f.read()

        val = cer(groundTruth, text)
        res.append(val)
        print(f'{fileName}: {val}')

    return res

In [22]:
CERs = calcCER(11,13)
print(mean(CERs))

011: 44.52662721893491
012: 78.02844531633154
013: 154.44015444015443
92.3317423251403


In [23]:
#calc binary file
CERs = calcCER(11,13, 'test_images/captured/binarized/..._SCANNED_BIN_GT.png')
print(mean(CERs))

011: 3.4023668639053253
012: 20.20598332515939
013: 136.87258687258688
53.493645687217196


In [27]:
#print text of one image
fileName = '012'
imageName = f'test_images/captured/grayscale/{fileName}_CC_GRAY.jpg'
binaryName = f'test_images/captured/binarized/{fileName}_SCANNED_BIN_GT.png'

print(img2text(binaryName))

CHAPTER 3. CONCEPT LEARNING FROM WEB VIDEO

MAP on validation set MAP on test set

10

08

  

23 FF 0.40 =

& 2 S 0.35

gz rot os 3

° as 0.30 0.90
os

a ass 028 0.25
C ™ 0.20 0.20

Q

o

00 02 04 O86 O8 10 00 02 04 O06 O8 1.0
welght{SIFT) welght(SIFT)

{a) (b)

Figure 3.11: The concept detection performance (MAP) of TubeTagger plotted against
the weights for SIFT+SVM and Motion. Similar performance can be observed on the
validation set (a) and on the test set (b), which indicates that feature weights can be
learned reliably.

mance for visual word approaches for the domain of news video [vdSGS08] — ob-
viously, similar observations hold for web video content. Exceptions are some con-
cepts for which the color+texture pipeline improves performance strongly. These
are mostly sports concepts, like “golf”, “basketball”, and “swimming”, for which
color is obviously a strong clue. Also, sports-related concepts reach the highest
overall performance, obviously because they come with a static

In [ ]:
# img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
edged = cv2.Canny(img, 100, 200)

cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

# loop over our contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.015 * peri, True)
	# if our approximated contour has four points, then
	# we can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break

cv2.drawContours(edged, [screenCnt], -1, (0, 255, 0), 3) 

cv2.imwrite('origin.jpg', edged)